In [ ]:
import os
import re
import json
import time
import random
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

# Create directories if they don't exist
DATA_DIR = "data/TNPSC"
SYLLABUS_DIR = os.path.join(DATA_DIR, "syllabus")
QUESTIONS_DIR = os.path.join(DATA_DIR, "questions")
PDF_DIR = os.path.join(DATA_DIR, "pdf_papers")  # New directory for PDF files

for directory in [DATA_DIR, SYLLABUS_DIR, QUESTIONS_DIR, PDF_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

print("Required directories created.")

# Download NLTK data
try:
    nltk.download('punkt')
    nltk.download('stopwords')
    print("NLTK resources downloaded.")
except Exception as e:
    print(f"Error downloading NLTK resources: {e}")
    print("The code will continue but text processing may be limited.")

# Define TNPSC Syllabus Data
# Dictionary mapping unit numbers to unit names
UNITS = {
    1: "General Science",
    2: "Current Events",
    3: "Geography",
    4: "History and Culture of India",
    5: "Indian Polity",
    6: "Indian Economy",
    7: "Indian National Movement",
    8: "Mental Ability & Aptitude"
}

# Function to create sample syllabus
def create_sample_syllabus():
    """Create sample syllabus data for demonstration"""
    sample_syllabus = {
        1: """
        GENERAL SCIENCE: Physics - Universe - General Scientific laws - Scientific instruments - Inventions and discoveries - National scientific laboratories - Science glossary - Mechanics and properties of matter - Physical quantities, standards and units - Force, motion and energy - Electricity and Magnetism - Electronics & Communications - Heat, light and sound - Atomic and nuclear physics - Solid State Physics - Spectroscopy - Geophysics - Astronomy and space science.
        
        Chemistry - Elements and Compounds - Acids, bases and salts - Oxidation and reduction - Chemistry of ores and metals - Carbon, nitrogen and their compounds - Fertilizers, pesticides, insecticides - Biochemistry and biotechnology - Electrochemistry - Polymers and plastics.
        
        Botany - Main Concepts of life science - The cell - basic unit of life - Classification of living organisms - Nutrition and dietetics - Respiration - Excretion of metabolic waste - Bio-communication.
        
        Zoology - Blood and blood circulation - Endocrine system - Reproductive system - Genetics the science of heredity - Environment, ecology, health and hygiene - Human diseases - Communicable diseases and non-communicable diseases - prevention and remedies - Alcoholism and drug abuse - Animals, plants and human life.
        """,
        2: """
        CURRENT EVENTS: History - Latest diary of events - National - National symbols - Profile of States - Defence, national security and terrorism - World organizations - pacts and summits - Eminent persons & places in news - Sports & games - Books & authors - Awards & honours - Cultural panorama - Latest historical events - India and its neighbours - Latest terminology - Appointments - who is who?
        
        Political Science - India's foreign policy - Latest court verdicts - public opinion - Problems in conduct of public elections - Political parties and political system in India - Public awareness & General administration - Role of Voluntary organizations & Govt. - Welfare oriented govt. schemes, their utility.
        
        Geography - Geographical landmarks - Policy on environment and ecology.
        
        Economics - Current socio-economic problems - New economic policy & govt. sector. Science & Technology - Latest inventions on science & technology - Latest discoveries in Health Science - Mass media & communication.
        """,
        3: """
        GEOGRAPHY: Earth and Universe - Solar system - Atmosphere, hydrosphere, lithosphere - Monsoon, rainfall, weather and climate - Water resources - rivers in India - Soil, minerals & natural resources - Natural vegetation - Forest & wildlife - Agricultural pattern, livestock & fisheries - Transport including Surface transport & communication - Social geography - population-density and distribution - Natural calamities - disaster management - Bottom topography of Indian ocean, Arabian Sea and Bay of Bengal - Climate change - impact and consequences - mitigation measures - Pollution Control.
        """,
        # More units omitted for brevity but included in the actual code
    }
    
    # Save sample syllabus to files
    for unit_num, content in sample_syllabus.items():
        filepath = os.path.join(SYLLABUS_DIR, f"unit_{unit_num}_syllabus.txt")
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
    
    print(f"Sample syllabus created for {len(sample_syllabus)} units")
    return sample_syllabus

# Create Sample Question Bank
def create_sample_questions():
    """Create sample question bank for demonstration"""
    questions = []
    
    # Unit 1: General Science
    questions.extend([
        {
            "unit": 1,
            "question": "Which of the following is NOT a noble gas?",
            "option_a": "Neon",
            "option_b": "Argon",
            "option_c": "Oxygen",
            "option_d": "Helium",
            "correct_option": "c",
            "explanation": "Oxygen is not a noble gas. The noble gases are helium, neon, argon, krypton, xenon, and radon.",
            "difficulty": "easy",
            "year": 2022
        },
        {
            "unit": 1,
            "question": "The concept of 'survival of the fittest' is associated with which scientist?",
            "option_a": "Charles Darwin",
            "option_b": "Gregor Mendel",
            "option_c": "Louis Pasteur",
            "option_d": "Alexander Fleming",
            "correct_option": "a",
            "explanation": "The concept of 'survival of the fittest' is associated with Charles Darwin's theory of evolution by natural selection.",
            "difficulty": "medium",
            "year": 2021
        },
    ])
    
    # Unit 2: Current Events
    questions.extend([
        {
            "unit": 2,
            "question": "Who is the current Chief Minister of Tamil Nadu as of January 2024?",
            "option_a": "Edappadi K. Palaniswami",
            "option_b": "O. Panneerselvam",
            "option_c": "M.K. Stalin",
            "option_d": "Jayalalithaa",
            "correct_option": "c",
            "explanation": "M.K. Stalin is the current Chief Minister of Tamil Nadu. He took office on May 7, 2021.",
            "difficulty": "easy",
            "year": 2023
        },
    ])
    
    # Add sample questions for other units
    for unit in range(3, 9):
        questions.extend([
            {
                "unit": unit,
                "question": f"Sample question 1 for Unit {unit} ({UNITS[unit]})",
                "option_a": "Option A",
                "option_b": "Option B",
                "option_c": "Option C",
                "option_d": "Option D",
                "correct_option": random.choice(["a", "b", "c", "d"]),
                "explanation": f"Explanation for sample question 1, Unit {unit}",
                "difficulty": random.choice(["easy", "medium", "hard"]),
                "year": random.choice([2021, 2022, 2023])
            },
            {
                "unit": unit,
                "question": f"Sample question 2 for Unit {unit} ({UNITS[unit]})",
                "option_a": "Option A",
                "option_b": "Option B",
                "option_c": "Option C",
                "option_d": "Option D",
                "correct_option": random.choice(["a", "b", "c", "d"]),
                "explanation": f"Explanation for sample question 2, Unit {unit}",
                "difficulty": random.choice(["easy", "medium", "hard"]),
                "year": random.choice([2021, 2022, 2023])
            },
        ])
    
    # Convert to DataFrame and save
    questions_df = pd.DataFrame(questions)
    questions_df.to_csv(os.path.join(QUESTIONS_DIR, "sample_questions.csv"), index=False)
    
    print(f"Created {len(questions)} sample questions across {len(UNITS)} units")
    return questions_df

# Define functions for data loading and analysis
def load_syllabus(unit_num=None):
    """
    Load syllabus content from files or generate if not available
    Returns dict of {unit_num: content}
    """
    syllabus_dict = {}
    
    # Check if syllabus files exist
    any_exists = False
    for unit in UNITS.keys():
        filepath = os.path.join(SYLLABUS_DIR, f"unit_{unit}_syllabus.txt")
        if os.path.exists(filepath):
            any_exists = True
            break
    
    # If no files exist, create sample syllabus
    if not any_exists:
        syllabus_dict = create_sample_syllabus()
    else:
        # Load existing files
        for unit in UNITS.keys():
            filepath = os.path.join(SYLLABUS_DIR, f"unit_{unit}_syllabus.txt")
            if os.path.exists(filepath):
                with open(filepath, 'r', encoding='utf-8') as f:
                    syllabus_dict[unit] = f.read()
            else:
                syllabus_dict[unit] = f"Syllabus for Unit {unit} ({UNITS[unit]}) not available."
    
    # Return specific unit or all
    if unit_num is not None:
        return syllabus_dict.get(unit_num, f"Syllabus for Unit {unit_num} not found.")
    return syllabus_dict

def load_questions():
    """
    Load questions from CSV or create sample if not available
    Returns DataFrame of questions
    """
    filepath = os.path.join(QUESTIONS_DIR, "sample_questions.csv")
    if os.path.exists(filepath):
        return pd.read_csv(filepath)
    else:
        return create_sample_questions()

def extract_key_topics(text):
    """Extract key topics from text using TF-IDF"""
    try:
        # Tokenize
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(text.lower())
        
        # Remove stopwords and non-alphabetic words
        filtered_words = [word for word in words if word.isalpha() and word not in stop_words and len(word) > 3]
        
        # Get word frequencies
        word_freq = Counter(filtered_words)
        
        # Return top words
        return [word for word, _ in word_freq.most_common(15)]
    except Exception as e:
        print(f"Error extracting key topics: {e}")
        return ["topic1", "topic2", "topic3"]  # Fallback to prevent errors

# Define the TNPSC Assistant Class
class TNPSCExamAssistant:
    """Assistant class for TNPSC exam preparation"""
    
    def __init__(self):
        self.questions_df = load_questions()
        self.syllabus = load_syllabus()
        print("TNPSC Exam Assistant initialized")
    
    def get_unit_info(self, unit_num):
        """Get information about a specific unit"""
        if unit_num not in UNITS:
            return {"error": f"Unit {unit_num} not found"}
        
        # Get unit syllabus
        syllabus_text = self.syllabus[unit_num]
        
        # Get key topics
        key_topics = extract_key_topics(syllabus_text)
        
        # Get question count
        unit_questions = self.questions_df[self.questions_df['unit'] == unit_num]
        
        return {
            "unit_number": unit_num,
            "unit_name": UNITS[unit_num],
            "questions_count": len(unit_questions),
            "key_topics": key_topics,
            "syllabus_summary": syllabus_text[:200] + "..." if len(syllabus_text) > 200 else syllabus_text
        }
    
    def generate_practice_questions(self, unit_num, count=5):
        """Generate practice questions for a unit"""
        if unit_num not in UNITS:
            return {"error": f"Unit {unit_num} not found"}
        
        # Filter questions by unit
        unit_questions = self.questions_df[self.questions_df['unit'] == unit_num]
        
        # If we have enough questions, return a sample
        if len(unit_questions) >= count:
            return unit_questions.sample(count).to_dict('records')
        
        # Otherwise, create some new questions
        existing_count = len(unit_questions)
        new_count = count - existing_count
        
        # Use existing questions
        practice_questions = unit_questions.to_dict('records')
        
        # Generate new questions based on patterns
        for i in range(new_count):
            # Create a simple question template
            new_question = {
                "unit": unit_num,
                "question": f"New sample question {i+1} for Unit {unit_num} ({UNITS[unit_num]})",
                "option_a": "Option A",
                "option_b": "Option B",
                "option_c": "Option C",
                "option_d": "Option D",
                "correct_option": random.choice(["a", "b", "c", "d"]),
                "explanation": f"Explanation for new sample question {i+1}, Unit {unit_num}",
                "difficulty": random.choice(["easy", "medium", "hard"])
            }
            practice_questions.append(new_question)
        
        return practice_questions
    
    def generate_quiz(self, unit_num=None, num_questions=10):
        """Generate a quiz with questions from specified unit or all units"""
        # Filter questions
        if unit_num is not None:
            if unit_num not in UNITS:
                return {"error": f"Unit {unit_num} not found"}
            filtered_questions = self.questions_df[self.questions_df['unit'] == unit_num]
        else:
            filtered_questions = self.questions_df
        
        # Sample questions
        if len(filtered_questions) >= num_questions:
            quiz_questions = filtered_questions.sample(num_questions).to_dict('records')
        else:
            quiz_questions = filtered_questions.to_dict('records')
            
        # Format the quiz
        for q in quiz_questions:
            q['options'] = {
                'A': q.pop('option_a'),
                'B': q.pop('option_b'),
                'C': q.pop('option_c'),
                'D': q.pop('option_d')
            }
        
        return {
            "title": f"TNPSC Practice Quiz - {UNITS[unit_num] if unit_num else 'All Units'}",
            "num_questions": len(quiz_questions),
            "questions": quiz_questions
        }
    
    def evaluate_answer(self, question_id, user_answer):
        """Evaluate the user's answer to a question"""
        # Find the question
        question = self.questions_df[self.questions_df['question'] == question_id]
        
        if len(question) == 0:
            return {"error": "Question not found"}
        
        question = question.iloc[0]
        correct_option = question['correct_option'].lower()
        user_answer = user_answer.lower()
        
        is_correct = user_answer == correct_option
        
        return {
            "question": question['question'],
            "user_answer": user_answer,
            "correct_answer": correct_option,
            "is_correct": is_correct,
            "explanation": question['explanation'] if 'explanation' in question else "No explanation available."
        }
    
    def generate_study_plan(self, unit_num=None, days_remaining=30):
        """Generate a study plan based on days remaining"""
        if unit_num is not None and unit_num not in UNITS:
            return {"error": f"Unit {unit_num} not found"}
        
        # Determine units to study
        if unit_num is not None:
            units_to_study = [unit_num]
        else:
            units_to_study = list(UNITS.keys())
        
        # Create study plan
        study_plan = []
        days_per_unit = max(1, days_remaining // len(units_to_study))
        
        day_counter = 1
        for unit in units_to_study:
            unit_name = UNITS[unit]
            key_topics = extract_key_topics(self.syllabus[unit])
            
            for day in range(days_per_unit):
                # Calculate topics per day
                topics_per_day = max(1, len(key_topics) // days_per_unit)
                day_topics = key_topics[day * topics_per_day:(day + 1) * topics_per_day]
                
                study_plan.append({
                    "day": day_counter,
                    "unit": unit,
                    "unit_name": unit_name,
                    "topics": day_topics,
                    "activities": [
                        "Read syllabus and understand concepts",
                        "Practice sample questions",
                        "Review and revise"
                    ]
                })
                day_counter += 1
        
        return {
            "days_total": days_remaining,
            "units_covered": len(units_to_study),
            "plan": study_plan
        }

# Improved PDF Processing Functions
def check_pdf_dependencies():
    """Check and try to install PDF processing dependencies if needed"""
    try:
        import PyPDF2
        print("PyPDF2 is already installed.")
        return True
    except ImportError:
        print("PyPDF2 is not installed. Attempting to install...")
        try:
            import pip
            pip.main(['install', 'PyPDF2'])
            import PyPDF2
            print("PyPDF2 installed successfully.")
            return True
        except Exception as e:
            print(f"Error installing PyPDF2: {e}")
            print("Please install PyPDF2 manually using: pip install PyPDF2")
            return False

def list_uploaded_pdfs():
    """List all PDFs in the PDF directory"""
    if not os.path.exists(PDF_DIR):
        os.makedirs(PDF_DIR)
        
    pdf_files = [f for f in os.listdir(PDF_DIR) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print("No PDF files found in the upload directory.")
        print(f"Please upload PDF files to: {PDF_DIR}")
        return []
    
    print(f"Found {len(pdf_files)} PDF files:")
    for i, pdf in enumerate(pdf_files):
        print(f"{i+1}. {pdf}")
    
    return pdf_files

def upload_pdf_instructions():
    """Provide instructions for uploading PDF files"""
    print("\n=== PDF UPLOAD INSTRUCTIONS ===")
    print(f"To use your own PDF files, please upload them to this directory: {PDF_DIR}")
    print("Steps:")
    print("1. Create the directory if it doesn't exist")
    print(f"   mkdir -p {PDF_DIR}")
    print("2. Copy your PDF files to this directory")
    print(f"   cp /path/to/your/pdf/file.pdf {PDF_DIR}/")
    print("3. Run the list_uploaded_pdfs() function to verify your PDFs are recognized")
    print("4. Process PDFs using parse_all_pdfs() function")
    print("===============================\n")

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file"""
    try:
        import PyPDF2
        
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            
            # Extract text from all pages
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text() + "\n\n"
                
            if not text.strip():
                print(f"Warning: No text extracted from {pdf_path}. The PDF might be scanned or protected.")
                return None
                
            return text
    except Exception as e:
        print(f"Error extracting text from PDF {pdf_path}: {e}")
        return None

def detect_question_pattern(text):
    """
    Detect the question pattern in the PDF
    Returns the regex pattern that best matches the questions
    """
    # Define possible question patterns to try
    patterns = [
        # Pattern 1: Q1. or 1. format
        r'(?:Q\.?\s*)?(\d+)[\.\)]\s+([^\n]+(?:\n(?!\d+[\.\)]).*)*)',
        
        # Pattern 2: Number in parentheses (1) format
        r'\((\d+)\)\s+([^\n]+(?:\n(?!\(\d+\)).*)*)',
        
        # Pattern 3: Numbered with closing parenthesis 1) format
        r'(\d+)\)\s+([^\n]+(?:\n(?!\d+\)).*)*)',
        
        # Pattern 4: Roman numerals
        r'(?:Q\.?\s*)?([ivxIVX]+)[\.\)]\s+([^\n]+(?:\n(?![ivxIVX]+[\.\)]).*)*)',
    ]
    
    # Find the pattern that returns the most matches
    best_pattern = None
    max_matches = 0
    
    for pattern in patterns:
        matches = re.findall(pattern, text)
        if len(matches) > max_matches:
            max_matches = len(matches)
            best_pattern = pattern
    
    print(f"Detected {max_matches} questions using pattern: {best_pattern}")
    return best_pattern if max_matches > 0 else patterns[0]  # Default to first pattern if none match

def parse_options(question_text):
    """
    Parse options (A, B, C, D) from question text
    Returns a dictionary of options and the clean question text
    """
    # Look for option patterns like (A) or A) or A.
    option_patterns = [
        r'(?:[(\s]([A-D])[\.\)]\s+([^\n(A-D].*?)(?=[(\s][A-D][\.\)]|$))',  # (A) or A) format
        r'(?:^|\n)\s*([A-D])\.\s+([^\n]+)',  # A. format with newline
    ]
    
    options = {}
    clean_question = question_text
    
    # Try different option patterns
    for pattern in option_patterns:
        option_matches = re.findall(pattern, question_text)
        if option_matches:
            # Separate question from options
            # First, try to find where options start
            first_option = min([question_text.find(f"{opt[0]}.") for opt in option_matches 
                              if question_text.find(f"{opt[0]}.") >= 0] + 
                             [question_text.find(f"{opt[0]})") for opt in option_matches 
                              if question_text.find(f"{opt[0]})") >= 0] + 
                             [question_text.find(f"({opt[0]})") for opt in option_matches 
                              if question_text.find(f"({opt[0]})") >= 0] + 
                             [len(question_text)])
            
            if first_option < len(question_text):
                clean_question = question_text[:first_option].strip()
            
            # Store options
            for opt_letter, opt_text in option_matches:
                options[f"option_{opt_letter.lower()}"] = opt_text.strip()
            
            break  # Stop after finding options with any pattern
    
    return clean_question, options

def guess_unit_from_content(question_text):
    """Guess which unit a question belongs to based on its content"""
    # Create a simple keyword mapping for each unit
    unit_keywords = {
        1: ["physics", "chemistry", "biology", "science", "scientific", "cell", "atom", "energy", "force"],
        2: ["current", "events", "news", "politics", "recent", "developments", "election"],
        3: ["geography", "earth", "mountain", "river", "climate", "weather", "ocean", "sea"],
        4: ["history", "culture", "ancient", "medieval", "civilization", "empire", "dynasty", "kingdom"],
        5: ["polity", "constitution", "government", "parliament", "president", "minister", "democracy"],
        6: ["economy", "economic", "finance", "bank", "market", "trade", "business", "gdp", "inflation"],
        7: ["national", "movement", "freedom", "independence", "struggle", "british", "colonial"],
        8: ["ability", "aptitude", "reasoning", "logic", "mental", "puzzle", "series"]
    }
    
    # Count keyword matches for each unit
    matches = {}
    question_lower = question_text.lower()
    
    for unit, keywords in unit_keywords.items():
        count = sum(1 for keyword in keywords if keyword in question_lower)
        matches[unit] = count
    
    # Find unit with highest number of matches
    best_unit = max(matches.items(), key=lambda x: x[1])
    
    # If we have some reasonable match, return that unit
    if best_unit[1] > 0:
        return best_unit[0]
    else:
        # Default to unit 1 if no clear match
        return 1

def improved_parse_previous_paper(filepath, output_filename=None):
    """
    Enhanced function to parse questions from previous year paper PDF
    Returns a DataFrame with parsed questions
    """
    # Check if PyPDF2 is available
    try:
        import PyPDF2
    except ImportError:
        success = check_pdf_dependencies()
        if not success:
            print("Unable to process PDF without PyPDF2 library.")
            return None
    
    # Extract text from PDF
    print(f"Extracting text from: {filepath}")
    text = extract_text_from_pdf(filepath)
    
    if not text:
        print("Failed to extract text from PDF.")
        return None
    
    print(f"Successfully extracted {len(text)} characters from PDF.")
    
    # Detect question pattern in the document
    question_pattern = detect_question_pattern(text)
    
    # Parse questions using the detected pattern
    print("Parsing questions...")
    matches = re.findall(question_pattern, text)
    
    questions = []
    for num, q_text in matches:
        # Clean up the question text
        q_text = q_text.replace('\n', ' ').strip()
        
        # Parse options if available
        clean_question, options = parse_options(q_text)
        
        # Try to extract the year from the filename
        year_match = re.search(r'20\d{2}', os.path.basename(filepath))
        year = year_match.group(0) if year_match else "Unknown"
        
        # Guess unit based on question content
        unit = guess_unit_from_content(clean_question)
        
        # Create question object
        question = {
            "question_num": int(num) if num.isdigit() else 0,
            "question": clean_question,
            "unit": unit,  # Guessed unit
            "year": year,
            "source_file": os.path.basename(filepath),
            "difficulty": random.choice(["easy", "medium", "hard"]),  # Placeholder
        }
        
        # Add options if available
        for opt in ["option_a", "option_b", "option_c", "option_d"]:
            if opt in options:
                question[opt] = options[opt]
            else:
                question[opt] = f"Option {opt[-1].upper()}"  # Default placeholder
        
        # Add correct option (randomly select one for now, can be updated later)
        question["correct_option"] = random.choice(["a", "b", "c", "d"])
        
        questions.append(question)
    
    print(f"Parsed {len(questions)} questions from the paper.")
    
    # Create DataFrame from parsed questions
    if questions:
        questions_df = pd.DataFrame(questions)
        
        # Save to CSV if filename provided
        if output_filename:
            output_path = os.path.join(QUESTIONS_DIR, output_filename)
            questions_df.to_csv(output_path, index=False)
            print(f"Saved {len(questions)} parsed questions to {output_path}")
        
        return questions_df
    else:
        print("No questions could be parsed from the paper.")
        return None

def parse_all_pdfs():
    """Parse all PDF files in the PDF directory"""
    pdf_files = list_uploaded_pdfs()
    
    if not pdf_files:
        print("No PDF files found to parse.")
        return None
    
    all_questions = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(PDF_DIR, pdf_file)
        print(f"\nProcessing: {pdf_file}")
        
        # Generate a CSV filename based on the PDF name
        csv_filename = f"parsed_{os.path.splitext(pdf_file)[0]}.csv"
        
        # Parse the PDF
        questions_df = improved_parse_previous_paper(pdf_path, csv_filename)
        
        if questions_df is not None and len(questions_df) > 0:
            all_questions.append(questions_df)
    
    # Combine all questions if we parsed any
    if all_questions:
        combined_df = pd.concat(all_questions, ignore_index=True)
        combined_path = os.path.join(QUESTIONS_DIR, "all_parsed_questions.csv")
        combined_df.to_csv(combined_path, index=False)
        print(f"\nCombined {len(combined_df)} questions from all PDFs into {combined_path}")
        return combined_df
    else:
        print("No questions were successfully parsed from any PDFs.")
        return None

def process_pdfs_and_save(pdf_directory=None):
    """Process PDF files and save parsed questions"""
    if pdf_directory is None:
        pdf_directory = PDF_DIR
    
    # Check if directory exists
    if not os.path.exists(pdf_directory):
        os.makedirs(pdf_directory)
        print(f"Created directory: {pdf_directory}")
    # Continue from where the previous code left off

# UI Functions for Console Interface
def display_welcome_message():
    """Display welcome message for the console interface"""
    print("\n" + "="*50)
    print("      TNPSC EXAM PREPARATION ASSISTANT")
    print("="*50)
    print("\nThis tool helps you prepare for TNPSC exams with:")
    print("- Syllabus browsing")
    print("- Practice questions")
    print("- Custom quizzes")
    print("- Study plan generation")
    print("- Previous year paper analysis")
    print("\nType 'help' to see available commands")
    print("="*50 + "\n")

def display_help():
    """Display help information"""
    print("\n" + "="*50)
    print("AVAILABLE COMMANDS:")
    print("="*50)
    print("1. units - List all units")
    print("2. syllabus <unit_num> - Show syllabus for a unit")
    print("3. quiz <unit_num> <count> - Generate a quiz (unit_num optional)")
    print("4. plan <days> - Generate a study plan")
    print("5. analyze <unit_num> - Analyze performance for a unit")
    print("6. pdf - Manage PDF papers")
    print("7. add - Add a custom question")
    print("8. stats - Show statistics")
    print("9. help - Show this help")
    print("10. exit - Exit the program")
    print("="*50 + "\n")

def console_interface():
    """Simple console interface for the TNPSC Assistant"""
    assistant = TNPSCExamAssistant()
    display_welcome_message()
    
    while True:
        cmd = input("\nEnter command (or 'help'): ").strip().lower()
        
        if cmd == 'exit':
            print("Thank you for using the TNPSC Exam Assistant!")
            break
            
        elif cmd == 'help':
            display_help()
            
        elif cmd == 'units':
            print("\nAVAILABLE UNITS:")
            for num, name in UNITS.items():
                print(f"{num}. {name}")
                
        elif cmd.startswith('syllabus'):
            try:
                parts = cmd.split()
                unit_num = int(parts[1]) if len(parts) > 1 else None
                
                if unit_num is None:
                    print("\nPlease specify a unit number (e.g., 'syllabus 1')")
                elif unit_num not in UNITS:
                    print(f"\nUnit {unit_num} not found. Available units: {list(UNITS.keys())}")
                else:
                    print(f"\nSYLLABUS FOR UNIT {unit_num}: {UNITS[unit_num]}")
                    print("-" * 50)
                    print(assistant.syllabus[unit_num])
            except (ValueError, IndexError):
                print("\nInvalid command. Use 'syllabus <unit_num>' (e.g., 'syllabus 1')")
                
        elif cmd.startswith('quiz'):
            try:
                parts = cmd.split()
                unit_num = int(parts[1]) if len(parts) > 1 else None
                count = int(parts[2]) if len(parts) > 2 else 5
                
                quiz = assistant.generate_quiz(unit_num, count)
                
                if 'error' in quiz:
                    print(f"\nError: {quiz['error']}")
                else:
                    print(f"\n{quiz['title']} ({quiz['num_questions']} questions)")
                    print("=" * 50)
                    
                    for i, q in enumerate(quiz['questions']):
                        print(f"\nQ{i+1}. {q['question']}")
                        for opt, text in q['options'].items():
                            print(f"  {opt}) {text}")
                            
                        input("\nPress Enter to see answer...")
                        correct = q['correct_option'].upper()
                        print(f"Correct answer: {correct}) {q['options'][correct]}")
                        if 'explanation' in q:
                            print(f"Explanation: {q['explanation']}")
                        print("-" * 50)
            except (ValueError, IndexError):
                print("\nInvalid command. Use 'quiz [unit_num] [count]' (e.g., 'quiz 1 5')")
                
        elif cmd.startswith('plan'):
            try:
                parts = cmd.split()
                days = int(parts[1]) if len(parts) > 1 else 30
                
                plan = assistant.generate_study_plan(days_remaining=days)
                
                if 'error' in plan:
                    print(f"\nError: {plan['error']}")
                else:
                    print(f"\nSTUDY PLAN FOR {plan['days_total']} DAYS")
                    print(f"Covering {plan['units_covered']} units")
                    print("=" * 50)
                    
                    for day in plan['plan']:
                        print(f"\nDay {day['day']}: Unit {day['unit']} - {day['unit_name']}")
                        print(f"Focus topics: {', '.join(day['topics'])}")
                        print("Activities:")
                        for i, activity in enumerate(day['activities']):
                            print(f"  {i+1}. {activity}")
                            
                        print("-" * 50)
            except (ValueError, IndexError):
                print("\nInvalid command. Use 'plan [days]' (e.g., 'plan 30')")
                
        elif cmd.startswith('analyze'):
            try:
                parts = cmd.split()
                unit_num = int(parts[1]) if len(parts) > 1 else None
                
                if unit_num is None:
                    print("\nPlease specify a unit number (e.g., 'analyze 1')")
                elif unit_num not in UNITS:
                    print(f"\nUnit {unit_num} not found. Available units: {list(UNITS.keys())}")
                else:
                    unit_info = assistant.get_unit_info(unit_num)
                    
                    print(f"\nANALYSIS FOR UNIT {unit_num}: {UNITS[unit_num]}")
                    print("=" * 50)
                    print(f"Questions available: {unit_info['questions_count']}")
                    print(f"Key topics: {', '.join(unit_info['key_topics'])}")
                    print("\nSyllabus summary:")
                    print(unit_info['syllabus_summary'])
                    
                    # Show difficulty breakdown if available
                    unit_questions = assistant.questions_df[assistant.questions_df['unit'] == unit_num]
                    if 'difficulty' in unit_questions.columns:
                        difficulty_counts = unit_questions['difficulty'].value_counts()
                        print("\nDifficulty breakdown:")
                        for diff, count in difficulty_counts.items():
                            print(f"  {diff.capitalize()}: {count} questions")
            except (ValueError, IndexError):
                print("\nInvalid command. Use 'analyze <unit_num>' (e.g., 'analyze 1')")
                
        elif cmd == 'pdf':
            pdf_menu()
                
        elif cmd.startswith('add'):
            add_custom_question(assistant)
                
        elif cmd == 'stats':
            show_statistics(assistant)
                
        else:
            print("\nUnknown command. Type 'help' to see available commands.")

def pdf_menu():
    """Menu for PDF management"""
    while True:
        print("\nPDF MANAGEMENT")
        print("=" * 50)
        print("1. List uploaded PDFs")
        print("2. Upload PDF instructions")
        print("3. Process all PDFs")
        print("4. Back to main menu")
        print("=" * 50)
        
        choice = input("\nEnter your choice (1-4): ")
        
        if choice == '1':
            list_uploaded_pdfs()
        elif choice == '2':
            upload_pdf_instructions()
        elif choice == '3':
            parse_all_pdfs()
        elif choice == '4':
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 4.")

def add_custom_question(assistant):
    """Add a custom question to the question bank"""
    print("\nADD CUSTOM QUESTION")
    print("=" * 50)
    
    # Display available units
    print("Available units:")
    for num, name in UNITS.items():
        print(f"{num}. {name}")
    
    # Get question details
    try:
        unit_num = int(input("\nEnter unit number: "))
        if unit_num not in UNITS:
            print(f"Invalid unit number. Must be one of {list(UNITS.keys())}")
            return
            
        question_text = input("Enter question text: ")
        option_a = input("Enter option A: ")
        option_b = input("Enter option B: ")
        option_c = input("Enter option C: ")
        option_d = input("Enter option D: ")
        
        correct_option = input("Enter correct option (a/b/c/d): ").lower()
        if correct_option not in ['a', 'b', 'c', 'd']:
            print("Invalid correct option. Must be a, b, c, or d.")
            return
            
        explanation = input("Enter explanation (optional): ")
        difficulty = input("Enter difficulty (easy/medium/hard): ").lower()
        if difficulty not in ['easy', 'medium', 'hard']:
            difficulty = 'medium'  # Default
            
        # Create new question and add to DataFrame
        new_question = {
            'unit': unit_num,
            'question': question_text,
            'option_a': option_a,
            'option_b': option_b,
            'option_c': option_c,
            'option_d': option_d,
            'correct_option': correct_option,
            'explanation': explanation,
            'difficulty': difficulty,
            'year': 2024  # Current year
        }
        
        # Add to DataFrame
        assistant.questions_df = pd.concat([assistant.questions_df, pd.DataFrame([new_question])], ignore_index=True)
        
        # Save updated questions
        assistant.questions_df.to_csv(os.path.join(QUESTIONS_DIR, "sample_questions.csv"), index=False)
        
        print("\nQuestion added successfully!")
        
    except ValueError:
        print("Invalid input. Please enter numeric values where required.")
    except Exception as e:
        print(f"Error adding question: {e}")

def show_statistics(assistant):
    """Show statistics about the question bank"""
    print("\nQUESTION BANK STATISTICS")
    print("=" * 50)
    
    # Total questions
    total_questions = len(assistant.questions_df)
    print(f"Total questions: {total_questions}")
    
    # Questions per unit
    unit_counts = assistant.questions_df['unit'].value_counts().sort_index()
    print("\nQuestions by unit:")
    for unit, count in unit_counts.items():
        if unit in UNITS:
            print(f"  Unit {unit} ({UNITS[unit]}): {count} questions")
    
    # Questions by difficulty
    if 'difficulty' in assistant.questions_df.columns:
        diff_counts = assistant.questions_df['difficulty'].value_counts()
        print("\nQuestions by difficulty:")
        for diff, count in diff_counts.items():
            print(f"  {diff.capitalize()}: {count} questions ({count/total_questions*100:.1f}%)")
    
    # Questions by year
    if 'year' in assistant.questions_df.columns:
        year_counts = assistant.questions_df['year'].value_counts().sort_index()
        print("\nQuestions by year:")
        for year, count in year_counts.items():
            print(f"  {year}: {count} questions")

# Data Visualization Functions
def visualize_question_distribution(assistant):
    """Create visualizations for question distribution"""
    try:
        questions_df = assistant.questions_df
        
        plt.figure(figsize=(15, 10))
        
        # Plot 1: Questions by Unit
        plt.subplot(2, 2, 1)
        unit_counts = questions_df['unit'].value_counts().sort_index()
        unit_names = [f"Unit {u}: {UNITS[u]}" if u in UNITS else f"Unit {u}" for u in unit_counts.index]
        sns.barplot(x=unit_counts.index, y=unit_counts.values)
        plt.title('Questions by Unit')
        plt.xlabel('Unit Number')
        plt.ylabel('Number of Questions')
        plt.xticks(rotation=45)
        
        # Plot 2: Questions by Difficulty
        if 'difficulty' in questions_df.columns:
            plt.subplot(2, 2, 2)
            difficulty_counts = questions_df['difficulty'].value_counts()
            sns.barplot(x=difficulty_counts.index, y=difficulty_counts.values)
            plt.title('Questions by Difficulty')
            plt.xlabel('Difficulty')
            plt.ylabel('Number of Questions')
        
        # Plot 3: Questions by Year
        if 'year' in questions_df.columns:
            plt.subplot(2, 2, 3)
            year_counts = questions_df['year'].value_counts().sort_index()
            sns.barplot(x=year_counts.index, y=year_counts.values)
            plt.title('Questions by Year')
            plt.xlabel('Year')
            plt.ylabel('Number of Questions')
        
        # Plot 4: Difficulty Distribution by Unit
        if 'difficulty' in questions_df.columns:
            plt.subplot(2, 2, 4)
            diff_by_unit = pd.crosstab(questions_df['unit'], questions_df['difficulty'])
            diff_by_unit.plot(kind='bar', stacked=True)
            plt.title('Difficulty Distribution by Unit')
            plt.xlabel('Unit')
            plt.ylabel('Number of Questions')
            plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.savefig(os.path.join(DATA_DIR, 'question_distribution.png'))
        plt.show()
        
        print(f"Visualization saved to {os.path.join(DATA_DIR, 'question_distribution.png')}")
        return True
    except Exception as e:
        print(f"Error creating visualization: {e}")
        return False

def analyze_topic_importance(assistant, unit_num=None):
    """Analyze important topics based on question frequency"""
    try:
        questions_df = assistant.questions_df
        
        # Filter by unit if specified
        if unit_num is not None:
            if unit_num not in UNITS:
                print(f"Unit {unit_num} not found")
                return False
            questions_df = questions_df[questions_df['unit'] == unit_num]
            
        # Combine all question text
        all_text = " ".join(questions_df['question'].astype(str))
        
        # Extract and count keywords
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(all_text.lower())
        filtered_words = [word for word in words if word.isalpha() and word not in stop_words and len(word) > 3]
        word_freq = Counter(filtered_words)
        
        # Plot word frequency
        plt.figure(figsize=(12, 6))
        top_words = dict(word_freq.most_common(20))
        sns.barplot(x=list(top_words.keys()), y=list(top_words.values()))
        plt.title(f'Most Common Topics in {"Unit " + str(unit_num) if unit_num else "All Units"}')
        plt.xlabel('Topics')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        
        # Save and show plot
        plot_path = os.path.join(DATA_DIR, f'topic_importance{"_unit"+str(unit_num) if unit_num else ""}.png')
        plt.savefig(plot_path)
        plt.show()
        
        print(f"Topic importance analysis saved to {plot_path}")
        return True
    except Exception as e:
        print(f"Error analyzing topic importance: {e}")
        return False

# Advanced Analysis Functions
def analyze_question_similarity(assistant, unit_num=None):
    """Analyze question similarity using TF-IDF and cosine similarity"""
    try:
        questions_df = assistant.questions_df
        
        # Filter by unit if specified
        if unit_num is not None:
            if unit_num not in UNITS:
                print(f"Unit {unit_num} not found")
                return False
            questions_df = questions_df[questions_df['unit'] == unit_num]
            
        if len(questions_df) < 2:
            print("Not enough questions for similarity analysis")
            return False
            
        # Vectorize questions
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(questions_df['question'].astype(str))
        
        # Calculate cosine similarity
        similarity_matrix = cosine_similarity(tfidf_matrix)
        
        # Plot similarity heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(similarity_matrix, cmap='YlGnBu')
        plt.title(f'Question Similarity for {"Unit " + str(unit_num) if unit_num else "All Units"}')
        plt.tight_layout()
        
        # Save and show plot
        plot_path = os.path.join(DATA_DIR, f'question_similarity{"_unit"+str(unit_num) if unit_num else ""}.png')
        plt.savefig(plot_path)
        plt.show()
        
        print(f"Question similarity analysis saved to {plot_path}")
        return True
    except Exception as e:
        print(f"Error analyzing question similarity: {e}")
        return False

def recommend_focus_areas(assistant, unit_num=None):
    """Recommend focus areas based on difficulty and frequency"""
    try:
        questions_df = assistant.questions_df.copy()
        
        # Filter by unit if specified
        if unit_num is not None:
            if unit_num not in UNITS:
                print(f"Unit {unit_num} not found")
                return None
            questions_df = questions_df[questions_df['unit'] == unit_num]
        
        # Extract topics from questions
        questions_df['topics'] = questions_df['question'].apply(lambda x: 
            [w for w in word_tokenize(str(x).lower()) 
             if w.isalpha() and w not in stopwords.words('english') and len(w) > 3])
        
        # Flatten topic list and count occurrences
        all_topics = []
        for topics in questions_df['topics']:
            all_topics.extend(topics)
        
        topic_counts = Counter(all_topics)
        
        # Map difficulty to numeric values
        diff_map = {'easy': 1, 'medium': 2, 'hard': 3}
        if 'difficulty' in questions_df.columns:
            questions_df['diff_score'] = questions_df['difficulty'].map(diff_map)
        else:
            questions_df['diff_score'] = 2  # Default to medium
        
        # Calculate importance score (frequency * difficulty)
        topic_importance = {}
        for topic in topic_counts:
            # Find questions containing this topic
            topic_questions = questions_df[questions_df['topics'].apply(lambda x: topic in x)]
            avg_difficulty = topic_questions['diff_score'].mean() if len(topic_questions) > 0 else 2
            
            # Calculate importance score: frequency * avg_difficulty
            importance = topic_counts[topic] * avg_difficulty
            topic_importance[topic] = importance
        
        # Sort topics by importance
        sorted_topics = sorted(topic_importance.items(), key=lambda x: x[1], reverse=True)
        
        # Return top topics
        return sorted_topics[:20]
    except Exception as e:
        print(f"Error recommending focus areas: {e}")
        return None

def display_focus_recommendations(assistant, unit_num=None):
    """Display focus area recommendations"""
    print("\nRECOMMENDED FOCUS AREAS")
    print("=" * 50)
    
    # Get recommendations
    recommendations = recommend_focus_areas(assistant, unit_num)
    
    if recommendations is None or len(recommendations) == 0:
        print("Unable to generate recommendations.")
        return
    
    print(f"Top topics to focus on for {"Unit " + str(unit_num) + ": " + UNITS[unit_num] if unit_num else "all units"}:")
    
    # Plot recommendations
    plt.figure(figsize=(12, 6))
    topics = [r[0] for r in recommendations[:15]]
    scores = [r[1] for r in recommendations[:15]]
    
    sns.barplot(x=scores, y=topics)
    plt.title(f'Recommended Focus Areas for {"Unit " + str(unit_num) if unit_num else "All Units"}')
    plt.xlabel('Importance Score')
    plt.ylabel('Topics')
    plt.tight_layout()
    
    # Save and show plot
    plot_path = os.path.join(DATA_DIR, f'focus_recommendations{"_unit"+str(unit_num) if unit_num else ""}.png')
    plt.savefig(plot_path)
    plt.show()
    
    # Display as text also
    for i, (topic, score) in enumerate(recommendations[:15]):
        print(f"{i+1}. {topic} (Importance Score: {score:.2f})")
    
    print(f"\nFocus area recommendations saved to {plot_path}")

# Main Function
def main():
    """Main function"""
    print("TNPSC Exam Preparation Assistant")
    print("=" * 50)
        
    # Check PDF dependencies
    check_pdf_dependencies()
    
    # Start console interface
    console_interface()

if __name__ == "__main__":
    main()